
Bryan Chi Fai Pang

Student ID: 501210081

TMU: The Chang School of Continuing Education

CIND 820 Big Data Analytics Project

Dr Ceni BABAOGLU

10 November 2023

#### Github Repository


https://github.com/bryantoca/capstone_project

Please note this notebook takse around 15 mintues to run / render.
An html version as well as NBViewer version are available on line.


# Initial Results and Code

## Part 1: Creation of Train, Validation, and Test Sets

1. **Train-Validate-Final Test Split:**

    - 80% train-validate split, 20% test set.
    - Anomaly records removed from train-validate sets.





2. **Handling Abnormal Records:**

    - Records with zero credit taken and grades throughout university but labeled as 'Graduate' moved.



3. **Target Mapping:**

    - Strings converted to numeric using the following mapping:
        {'Dropout': 0, 'Enrolled': 1, 'Graduate': 2}


## Part 2: Model Selection Using 10-fold Cross Validation

Using the following models:
- Logistic Regression
- Random Forest (n_estimators=10 and n_estimators=100)
- Support Vector Machine (SVM)
- SVM with a linear kernel
- Gradient Boosting
- XGB Classifier

Performance Metrics:
- Average Accuracy
- Average F1 Score
- Standard Deviation (SD)

## Part 3: Research Question Analysis

Subsets of the dataset are trained and tested using Random Forest and XGB Classifier with 80-20 train-validate split.

Subsets:
1. S1: Academic, Macroeconomic
2. S2: Academic, Macroeconomics, Demographic
3. S3: Academic, Macroeconomics, Socioeconomic
4. S4: Academic, Macroeconomic, Demographic, Socioeconomic
5. S5: Demographic, Socioeconomic

Full Classification Reports are displayed for each subset.


## Part 1: Creation of Train, Validation, and Test Sets

In [1]:
# import the necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.font_manager as fm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier  
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from statistics import mean, stdev
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix












In [2]:
# Using the datetime.now() at the beginning and at the end to check time 
# needed to run the codes.

# datetime object containing current date and time
Start = datetime.now()
 
print("Notebook started at ", Start )



Notebook started at  2023-11-22 16:18:54.072282


1. **Train-Validate-Final Test Split:**
    - 80% train-validate split, 20% test set (this test set is not use but for final test /results)
    

In [3]:
# creating train_valide and test_set 
# test_set is for final testing

data = pd.read_csv("data.csv",sep=";") 

#data = pd.read_csv("data_cat.csv", sep=";")

#train_validate_set, test_set = train_test_split(df, test_size = 0.2, random_state=76)

- Anomaly Detection : records removed from train-validate sets.
These records are students that are classified as "Graduate", but with zero for all curricular units taken as well as zero as grades. 


In [4]:

anomalies = data[(data['Target'] == 'Graduate') & (data.iloc[:, 21:33].eq(0).all(axis=1))]
anomalies_to_print = anomalies[["Target"]+list(data.columns[21:33])]
anomalies_to_print.head()



,Target,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations)
20,Graduate,0,0,0,0,0.0,0,0,0,0,0,0.0,0
66,Graduate,0,0,0,0,0.0,0,0,0,0,0,0.0,0
101,Graduate,0,0,0,0,0.0,0,0,0,0,0,0.0,0
405,Graduate,0,0,0,0,0.0,0,0,0,0,0,0.0,0
527,Graduate,0,0,0,0,0.0,0,0,0,0,0,0.0,0


In [5]:
anomal = data[(data['Target'] == 'Graduate') & (data.iloc[:, 21:33].eq(0).all(axis=1))]
anomal.index

Int64Index([  20,   66,  101,  405,  527,  534,  557,  574,  679,  722,  728,
             789,  821,  869,  881, 1002, 1050, 1192, 1302, 1350, 1363, 1377,
            1425, 1507, 1512, 1575, 1585, 1600, 1658, 1751, 1883, 1889, 1890,
            1898, 2008, 2026, 2124, 2143, 2175, 2194, 2230, 2235, 2328, 2356,
            2371, 2387, 2406, 2496, 2508, 2637, 2656, 2793, 2814, 2899, 2920,
            2955, 3023, 3024, 3135, 3160, 3317, 3405, 3447, 3481, 3683, 3707,
            3717, 3732, 3745, 3928, 3946, 4291, 4353, 4365, 4370],
           dtype='int64')

In [6]:
data.drop(anomal.index,inplace=True)

3. **Target Mapping:**
    - Strings converted to numeric using the following mapping:
        {'Dropout': 0, 'Enrolled': 1, 'Graduate': 2}

In [7]:
data_counts = data['Target'].value_counts()

print(data_counts)

Graduate    2134
Dropout     1421
Enrolled     794
Name: Target, dtype: int64


In [8]:




# Create a mapping dicctionary for Target, as XGBoost take only numeric value.
mapping = {'Dropout':0, 'Enrolled':0, 'Graduate':1}

data['Target']=data['Target'].map(mapping)


data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4349 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4349 non-null   int64  
 1   Application mode                                4349 non-null   int64  
 2   Application order                               4349 non-null   int64  
 3   Course                                          4349 non-null   int64  
 4   Daytime evening attendance                      4349 non-null   int64  
 5   Previous qualification                          4349 non-null   int64  
 6   Previous qualification (grade)                  4349 non-null   float64
 7   Nacionality                                     4349 non-null   int64  
 8   Mother's qualification                          4349 non-null   int64  
 9   Father's qualification                   

## Part 2: Model Selection Using 10-fold Cross Validation

Using the following models:

- Random Forest (n_estimators=10 and n_estimators=100)
- Support Vector Machine (SVM)
- SVM with a linear kernel
- Gradient Boosting
- XGB Classifier

Performance Metrics:
- Average Accuracy
- Average F1 Score
- Standard Deviation (SD)

Using 10-fold cross validation to give an initial results to see how different algorithms.

First, for each algorithm, F1 and Accuracy  is reported of each fold and then average F1 and Accuracy, as well as SD are reported. 

Summary of these models 

#### Results

Of the 6 models trained, Random Forest (n_estimator=100), Gradient Boosting and XGB Classifer have the highest accuracy and F1 score.

Based on these results, the project will focus of these three models and continue with  feature selection / feature engineering as well 
model hyperparametr tuning.



## Part 3: Research Question Analysis

Research question:

Dose inclusion of Demographic and Socioeconmoic data improve dropout prediction, and by how much ?
To annsewer the question, subsets of datasets are created and for initial results, Random Forest and XGB Classifier are used.

For each subset, Classification Report, Feature Importance and Permutation Importance are showned.

Subsets of the dataset are trained and tested using Random Forest and XGB Classifier with 80-20 train-validate split.

Subsets:
1. S1: Academic, Macroeconomic
2. S2: Academic, Macroeconomics, Demographic
3. S3: Academic, Macroeconomics, Socioeconomic
4. S4: Academic, Macroeconomic, Demographic, Socioeconomic
5. S5: Demographic, Socioeconomic

Full Classification Reports are displayed for each subset.

Creating the subsets

### Demographic Columns
- Marital status
- Nacionality
- Displaced
- Gender
- Age at enrollment
- International

### Socioeconomic Columns
- Mother's qualification
- Father's qualification
- Mother's occupation
- Father's occupation
- Educational special needs
- Debtor
- Tuition fees up to date
- Scholarship holder

### Macroeconomic Columns
- Unemployment rate
- Inflation rate
- GDP

### Academic Columns
- Application mode
- Application order
- Course
- Daytime evening attendance
- Previous qualification
- Previous qualification (grade)
- Admission grade
- Curricular units 1st sem (credited)
- Curricular units 1st sem (enrolled)
- Curricular units 1st sem (evaluations)
- Curricular units 1st sem (approved)
- Curricular units 1st sem (grade)
- Curricular units 1st sem (without evaluations)
- Curricular units 2nd sem (credited)
- Curricular units 2nd sem (enrolled)
- Curricular units 2nd sem (evaluations)
- Curricular units 2nd sem (approved)
- Curricular units 2nd sem (grade)
- Curricular units 2nd sem (without evaluations)

### Target Column
- Target


In [9]:
demographic_columns = ['Marital status','Nacionality','Displaced','Gender','Age at enrollment','International']
socioeconomic_columns =["Mother's qualification","Father's qualification","Mother's occupation","Father's occupation",'Educational special needs','Debtor',
 'Tuition fees up to date','Scholarship holder']
macroeconomic_columns = ['Unemployment rate','Inflation rate','GDP']
academic_columns = [
 'Application mode',
 'Application order',
 'Course',
 'Daytime evening attendance',
 'Previous qualification',
 'Previous qualification (grade)',
 'Admission grade',
 'Curricular units 1st sem (credited)',
 'Curricular units 1st sem (enrolled)',
 'Curricular units 1st sem (evaluations)',
 'Curricular units 1st sem (approved)',
 'Curricular units 1st sem (grade)',
 'Curricular units 1st sem (without evaluations)',
 'Curricular units 2nd sem (credited)',
 'Curricular units 2nd sem (enrolled)',
 'Curricular units 2nd sem (evaluations)',
 'Curricular units 2nd sem (approved)',
 'Curricular units 2nd sem (grade)',
 'Curricular units 2nd sem (without evaluations)'
 ]
target_s = ['Target']

In [10]:
s1 = data[target_s + macroeconomic_columns + academic_columns]
s2 = data[target_s + macroeconomic_columns + academic_columns + demographic_columns]
s3 = data[target_s + macroeconomic_columns + academic_columns + socioeconomic_columns]
s4 = data[target_s + macroeconomic_columns + academic_columns + socioeconomic_columns + demographic_columns]
s5 = data[target_s + demographic_columns + socioeconomic_columns ]

The 5 datasets with Random Forest

In [11]:


dataframes = [s1, s2, s3, s4, s5]  

attribute_groups = [
    ['Academic', 'Macroeconomic'],
    ['Academic', 'Macroeconomics', 'Demographic'],
    ['Academic', 'Macroeconomics', 'Socioeconomic'],
    ['Academic', 'Macroeconomic', 'Demographic', 'Socioeconomic'],
    ['Demographic', 'Socioeconomic']
]


results = []

for i, df in enumerate(dataframes, 1):
    target = df["Target"]
    features = df.drop("Target", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=76)




    
       



    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=76)
    rf_classifier.fit(X_train, y_train)
    predictions = rf_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    target_names = ['No Graduate', 'Graduate']
    report = classification_report(y_test, predictions, target_names=target_names)
    results.append((accuracy, report))

    print(f"Results for s{i}:")
    print(f"Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    print(f"Accuracy: {accuracy:.4f}")
    print(report)

    """

    # Confusion Matrix without normalization
    cm = confusion_matrix(y_test, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix without Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Confusion Matrix with normalization
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix with Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()


    feature_importance = rf_classifier.feature_importances_
    feature_names = list(X_train.columns)

    # Create a DataFrame with feature names and their importances
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Plotting feature importances
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(feature_importance_df)), feature_importance_df['Importance'], align='center')
    plt.xticks(range(len(feature_importance_df)), feature_importance_df['Feature'], rotation='vertical')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title(f'Random Forest Feature Importance for s{i} - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.tight_layout()
    plt.show()

    result = permutation_importance(rf_classifier, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

    sorted_importances_idx = result.importances_mean.argsort()
    importances = pd.DataFrame(
        result.importances[sorted_importances_idx].T,
        columns=X_train.columns[sorted_importances_idx],
    )

    ax = importances.plot.box(vert=False, whis=10)
    ax.set_title(f"Permutation Importances (test set) - Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    ax.axvline(x=0, color="k", linestyle="--")
    ax.set_xlabel("Decrease in accuracy score")
    ax.figure.tight_layout()
    plt.show()
    
"""
    print("-------------------------------------------------------------\n")


Results for s1:
Attribute Groups: Academic, Macroeconomic
Accuracy: 0.8529
              precision    recall  f1-score   support

 No Graduate       0.88      0.83      0.86       457
    Graduate       0.82      0.88      0.85       413

    accuracy                           0.85       870
   macro avg       0.85      0.85      0.85       870
weighted avg       0.85      0.85      0.85       870

-------------------------------------------------------------



Results for s2:
Attribute Groups: Academic, Macroeconomics, Demographic
Accuracy: 0.8552
              precision    recall  f1-score   support

 No Graduate       0.89      0.82      0.86       457
    Graduate       0.82      0.89      0.85       413

    accuracy                           0.86       870
   macro avg       0.86      0.86      0.86       870
weighted avg       0.86      0.86      0.86       870

-------------------------------------------------------------

Results for s3:
Attribute Groups: Academic, Macroeconomics, Socioeconomic
Accuracy: 0.8575
              precision    recall  f1-score   support

 No Graduate       0.89      0.83      0.86       457
    Graduate       0.83      0.89      0.86       413

    accuracy                           0.86       870
   macro avg       0.86      0.86      0.86       870
weighted avg       0.86      0.86      0.86       870

-------------------------------------------------------------

Results for s4:
Attribute Groups: Academ

The 5 datasets with Gradient Booster Classifer 

In [12]:
dataframes = [s1, s2, s3, s4, s5]  

attribute_groups = [
    ['Academic', 'Macroeconomic'],
    ['Academic', 'Macroeconomics', 'Demographic'],
    ['Academic', 'Macroeconomics', 'Socioeconomic'],
    ['Academic', 'Macroeconomic', 'Demographic', 'Socioeconomic'],
    ['Demographic', 'Socioeconomic']
]




results = []
for i, df in enumerate(dataframes, 1):
    target = df["Target"]
    features = df.drop("Target", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=76)
 
    GB_classifier = GradientBoostingClassifier( random_state=76)
    GB_classifier.fit(X_train, y_train)
    predictions = GB_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    target_names = [ 'No Graduate', 'Graduate']
    report = classification_report(y_test, predictions, target_names=target_names)
    results.append((accuracy, report))

    print(f"Results for s{i}:")
    print(f"Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    print(f"Accuracy: {accuracy:.4f}")
    print(report)

    """

    # Confusion Matrix without normalization
    cm = confusion_matrix(y_test, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix without Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Confusion Matrix with normalization
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix with Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    feature_importance = GB_classifier.feature_importances_
    feature_names = list(X_train.columns)

    # Create a DataFrame with feature names and their importances
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Plotting feature importances
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(feature_importance_df)), feature_importance_df['Importance'], align='center')
    plt.xticks(range(len(feature_importance_df)), feature_importance_df['Feature'], rotation='vertical')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title(f'GB Feature Importance for s{i} - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.tight_layout()
    plt.show()

    result = permutation_importance(GB_classifier, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

    sorted_importances_idx = result.importances_mean.argsort()
    importances = pd.DataFrame(
        result.importances[sorted_importances_idx].T,
        columns=X_train.columns[sorted_importances_idx],
    )

    ax = importances.plot.box(vert=False, whis=10)
    ax.set_title(f"Permutation Importances (test set) - Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    ax.axvline(x=0, color="k", linestyle="--")
    ax.set_xlabel("Decrease in accuracy score")
    ax.figure.tight_layout()
    plt.show()

    """
    

    print("-------------------------------------------------------------\n")


Results for s1:
Attribute Groups: Academic, Macroeconomic
Accuracy: 0.8563
              precision    recall  f1-score   support

 No Graduate       0.90      0.81      0.86       457
    Graduate       0.81      0.90      0.86       413

    accuracy                           0.86       870
   macro avg       0.86      0.86      0.86       870
weighted avg       0.86      0.86      0.86       870

-------------------------------------------------------------

Results for s2:
Attribute Groups: Academic, Macroeconomics, Demographic
Accuracy: 0.8552
              precision    recall  f1-score   support

 No Graduate       0.90      0.81      0.85       457
    Graduate       0.81      0.90      0.86       413

    accuracy                           0.86       870
   macro avg       0.86      0.86      0.86       870
weighted avg       0.86      0.86      0.86       870

-------------------------------------------------------------

Results for s3:
Attribute Groups: Academic, Macroeconomi

The 5 datasets with XGB Classifier



In [13]:


dataframes = [s1, s2, s3, s4, s5]  # 

attribute_groups = [
    ['Academic', 'Macroeconomic'],
    ['Academic', 'Macroeconomics', 'Demographic'],
    ['Academic', 'Macroeconomics', 'Socioeconomic'],
    ['Academic', 'Macroeconomic', 'Demographic', 'Socioeconomic'],
    ['Demographic', 'Socioeconomic']
]


results = []
for i, df in enumerate(dataframes, 1):
    target = df["Target"]
    features = df.drop("Target", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=76)
    xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=2, random_state=76)
    xgb_classifier.fit(X_train, y_train)
    predictions = xgb_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    target_names = ['Not Graduate', 'Graduate']
    report = classification_report(y_test, predictions, target_names=target_names)
    results.append((accuracy, report))

    print(f"Results for s{i}:")
    print(f"Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    print(f"Accuracy: {accuracy:.4f}")
    print(report)

    """

    # Confusion Matrix without normalization
    cm = confusion_matrix(y_test, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix without Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Confusion Matrix with normalization
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix with Normalization - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Feature Importance
    feature_importance = xgb_classifier.feature_importances_
    feature_names = list(X_train.columns)

    # Create a DataFrame with feature names and their importances
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Plotting feature importances
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(feature_importance_df)), feature_importance_df['Importance'], align='center')
    plt.xticks(range(len(feature_importance_df)), feature_importance_df['Feature'], rotation='vertical')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title(f'XGB Classifier Feature Importance for s{i} - Attribute Groups: {", ".join(attribute_groups[i - 1])}')
    plt.tight_layout()
    plt.show()

    # Permutation Importance
    result = permutation_importance(xgb_classifier, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
    sorted_importances_idx = result.importances_mean.argsort()
    importances = pd.DataFrame(
        result.importances[sorted_importances_idx].T,
        columns=X_test.columns[sorted_importances_idx],
    )

    ax = importances.plot.box(vert=False, whis=10)
    ax.set_title(f"Permutation Importances (test set) - Attribute Groups: {', '.join(attribute_groups[i - 1])}")
    ax.axvline(x=0, color="k", linestyle="--")
    ax.set_xlabel("Decrease in accuracy score")
    ax.figure.tight_layout()
    plt.show()
    """

    

    print("-------------------------------------------------------------\n")


Results for s1:
Attribute Groups: Academic, Macroeconomic
Accuracy: 0.8483
              precision    recall  f1-score   support

Not Graduate       0.87      0.83      0.85       457
    Graduate       0.82      0.87      0.84       413

    accuracy                           0.85       870
   macro avg       0.85      0.85      0.85       870
weighted avg       0.85      0.85      0.85       870

-------------------------------------------------------------

Results for s2:
Attribute Groups: Academic, Macroeconomics, Demographic
Accuracy: 0.8471
              precision    recall  f1-score   support

Not Graduate       0.88      0.82      0.85       457
    Graduate       0.81      0.88      0.85       413

    accuracy                           0.85       870
   macro avg       0.85      0.85      0.85       870
weighted avg       0.85      0.85      0.85       870

-------------------------------------------------------------

Results for s3:
Attribute Groups: Academic, Macroeconomi

Model Random Forest


### Summary of Confusion Matrices Model Random Forest

|  Subset  | Attribute Groups                              | Accuracy | Precision (No Graduate) | Recall (No Graduate) | F1-Score (No Graduate) | Precision (Graduate) | Recall (Graduate) | F1-Score (Graduate) | Macro Avg | Weighted Avg |
|----|-----------------------------------------------|----------|-------------------------|----------------------|-------------------------|----------------------|-------------------|----------------------|-----------|--------------|
| s1 | Academic, Macroeconomic                       | 0.8529   | 0.88                    | 0.83                 | 0.86                    | 0.82                 | 0.88              | 0.85                 | 0.85      | 0.85         |
| s2 | Academic, Macroeconomics, Demographic          | 0.8552   | 0.89                    | 0.82                 | 0.86                    | 0.82                 | 0.89              | 0.85                 | 0.86      | 0.86         |
| s3 | Academic, Macroeconomics, Socioeconomic        | 0.8575   | 0.89                    | 0.83                 | 0.86                    | 0.83                 | 0.89              | 0.86                 | 0.86      | 0.86         |
| s4 | Academic, Macroeconomic, Demographic, Socioeconomic | 0.8563   | 0.88                    | 0.84                 | 0.86                    | 0.83                 | 0.88              | 0.85                 | 0.86      | 0.86         |
| s5 | Demographic, Socioeconomic                    | 0.6586   | 0.68                    | 0.66                 | 0.67                    | 0.64                 | 0.66              | 0.65                 | 0.66      | 0.66         |




Model Gardien Boosting


### Summary of Confusion Matrices Model Gardien Boosting

|  Subset  | Attribute Groups                              | Accuracy | Precision (No Graduate) | Recall (No Graduate) | F1-Score (No Graduate) | Precision (Graduate) | Recall (Graduate) | F1-Score (Graduate) | Macro Avg | Weighted Avg |
|----|-----------------------------------------------|----------|-------------------------|----------------------|-------------------------|----------------------|-------------------|----------------------|-----------|--------------|
| s1 | Academic, Macroeconomic                       | 0.8563   | 0.90                    | 0.81                 | 0.86                    | 0.81                 | 0.90              | 0.86                 | 0.86      | 0.86         |
| s2 | Academic, Macroeconomics, Demographic          | 0.8552   | 0.90                    | 0.81                 | 0.85                    | 0.81                 | 0.90              | 0.86                 | 0.86      | 0.86         |
| s3 | Academic, Macroeconomics, Socioeconomic        | 0.8563   | 0.89                    | 0.83                 | 0.86                    | 0.83                 | 0.88              | 0.85                 | 0.86      | 0.86         |
| s4 | Academic, Macroeconomic, Demographic, Socioeconomic | 0.8540   | 0.89                    | 0.83                 | 0.86                    | 0.82                 | 0.88              | 0.85                 | 0.85      | 0.85         |
| s5 | Demographic, Socioeconomic                    | 0.7011   | 0.72                    | 0.70                 | 0.71                    | 0.68                 | 0.70              | 0.69                 | 0.70      | 0.70         |



Model XGBoost

### Summary of Confusion Matrices Model XGBoost

|  Subset  | Attribute Groups                              | Accuracy | Precision (Not Graduate) | Recall (Not Graduate) | F1-Score (Not Graduate) | Precision (Graduate) | Recall (Graduate) | F1-Score (Graduate) | Macro Avg | Weighted Avg |
|----|-----------------------------------------------|----------|--------------------------|-----------------------|--------------------------|-----------------------|-------------------|----------------------|-----------|--------------|
| s1 | Academic, Macroeconomic                       | 0.8483   | 0.87                     | 0.83                  | 0.85                     | 0.82                  | 0.87              | 0.84                 | 0.85      | 0.85         |
| s2 | Academic, Macroeconomics, Demographic          | 0.8471   | 0.88                     | 0.82                  | 0.85                     | 0.81                  | 0.88              | 0.85                 | 0.85      | 0.85         |
| s3 | Academic, Macroeconomics, Socioeconomic        | 0.8621   | 0.89                     | 0.84                  | 0.86                     | 0.83                  | 0.89              | 0.86                 | 0.86      | 0.86         |
| s4 | Academic, Macroeconomic, Demographic, Socioeconomic | 0.8471   | 0.87                     | 0.83                  | 0.85                     | 0.82                  | 0.86              | 0.84                 | 0.85      | 0.85         |
| s5 | Demographic, Socioeconomic                    | 0.6920   | 0.73                     | 0.66                  | 0.69                     | 0.66                  | 0.73              | 0.69                 | 0.69      | 0.70         |


In [14]:
end = datetime.now()
 
print("Notebook ended at ", end )


Notebook ended at  2023-11-22 16:19:09.898706
